In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("NBA Player Injury Stats(1951 - 2025).csv")
del df['Unnamed: 0']

In [5]:
body_injury = pd.read_csv("body_parts_and_injury_types.csv")

In [7]:
body_injury

,Body Parts,Injury Types
0,Ankle,Sprain
1,Knee,Strain
2,Foot,Fracture
3,Calf,Tear
4,Hamstring,Tendinitis
5,Quadriceps,Tendinopathy
6,Achilles,Dislocation
7,Toe,Bursitis
8,Shoulder,Spasms
9,Wrist,Stinger


In [9]:
df

,Date,Team,Acquired,Relinquished,Notes
0,1951-12-25,Bullets,NaN,Don Barksdale,placed on IL
1,1952-12-26,Knicks,NaN,Max Zaslofsky,placed on IL with torn side muscle
2,1956-12-29,Knicks,NaN,Jim Baechtold,placed on inactive list
3,1959-01-16,Lakers,NaN,Elgin Baylor,player refused to play after being denied a ro...
4,1961-11-26,Lakers,NaN,Elgin Baylor,player reported for military duty
...,...,...,...,...,...
40726,2025-01-12,Jazz,Brice Sensabaugh,NaN,activated from IL
40727,2025-01-12,Nets,Dariq Whitehead,NaN,activated from IL
40728,2025-01-12,Nuggets,Aaron Gordon,NaN,activated from IL
40729,2025-01-12,Suns,Royce O'Neale,NaN,activated from IL


In [15]:
import pandas as pd
inj = pd.read_csv("NBA Player Injury Stats(1951 - 2025).csv", parse_dates=["Date"]).rename(columns={"Date": "injury_date", "Notes": "notes"})

In [17]:
del inj['Unnamed: 0']
inj

,injury_date,Team,Acquired,Relinquished,notes
0,1951-12-25,Bullets,NaN,Don Barksdale,placed on IL
1,1952-12-26,Knicks,NaN,Max Zaslofsky,placed on IL with torn side muscle
2,1956-12-29,Knicks,NaN,Jim Baechtold,placed on inactive list
3,1959-01-16,Lakers,NaN,Elgin Baylor,player refused to play after being denied a ro...
4,1961-11-26,Lakers,NaN,Elgin Baylor,player reported for military duty
...,...,...,...,...,...
40726,2025-01-12,Jazz,Brice Sensabaugh,NaN,activated from IL
40727,2025-01-12,Nets,Dariq Whitehead,NaN,activated from IL
40728,2025-01-12,Nuggets,Aaron Gordon,NaN,activated from IL
40729,2025-01-12,Suns,Royce O'Neale,NaN,activated from IL


In [27]:
inj['notes'] = inj['notes'].astype('str')

In [29]:
# 2. Identify player involved and event type
def extract_player(row):
    txt = row["notes"].lower()
    if "placed on il" in txt or "placed on inactive" in txt:
        return row["Relinquished"]      # adjust column name to your CSV
    elif "activated" in txt:
        return row["Acquired"]     # adjust column name to your CSV
    else:
        return None

inj["player_name"] = inj.apply(extract_player, axis=1)
inj = inj.dropna(subset=["player_name"])

In [31]:
inj

,injury_date,Team,Acquired,Relinquished,notes,player_name
0,1951-12-25,Bullets,NaN,Don Barksdale,placed on IL,Don Barksdale
1,1952-12-26,Knicks,NaN,Max Zaslofsky,placed on IL with torn side muscle,Max Zaslofsky
2,1956-12-29,Knicks,NaN,Jim Baechtold,placed on inactive list,Jim Baechtold
8,1962-11-06,Zephyrs,Al Ferrari,NaN,activated from disabled list,Al Ferrari
11,1969-10-15,Bucks,NaN,Bob Greacen,placed on IL with sprained ankle,Bob Greacen
...,...,...,...,...,...,...
40726,2025-01-12,Jazz,Brice Sensabaugh,NaN,activated from IL,Brice Sensabaugh
40727,2025-01-12,Nets,Dariq Whitehead,NaN,activated from IL,Dariq Whitehead
40728,2025-01-12,Nuggets,Aaron Gordon,NaN,activated from IL,Aaron Gordon
40729,2025-01-12,Suns,Royce O'Neale,NaN,activated from IL,Royce O'Neale


In [33]:
# 3. Define body parts & injury types
body_parts   = ["Ankle","Knee","Foot","Calf","Hamstring","Quadriceps","Achilles","Toe",
                "Shoulder","Wrist","Finger","Elbow","Back","Neck","Head"]
injury_types = ["Sprain","Strain","Fracture","Tear","Tendinitis","Tendinopathy",
                "Dislocation","Bursitis","Spasms","Stinger","Contusion",
                "Concussion","Soreness","Inflammation","Surgery"]

# 4. One-hot encoding of each term in notes
for part in body_parts:
    inj[f"bp_{part}"] = inj["notes"].str.contains(part, case=False, na=False).astype(int)
for itype in injury_types:
    inj[f"it_{itype}"] = inj["notes"].str.contains(itype, case=False, na=False).astype(int)

/var/folders/y_/grgdfgkn6ydcldp4b25ckwwr0000gn/T/ipykernel_44935/1337989108.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj[f"bp_{part}"] = inj["notes"].str.contains(part, case=False, na=False).astype(int)
/var/folders/y_/grgdfgkn6ydcldp4b25ckwwr0000gn/T/ipykernel_44935/1337989108.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj[f"bp_{part}"] = inj["notes"].str.contains(part, case=False, na=False).astype(int)
/var/folders/y_/grgdfgkn6ydcldp4b25ckwwr0000gn/T/ipykernel_44935/1337989108.py:10:

In [35]:
# 5. Load season start dates
season_df = pd.read_csv("season_start_dates.csv", parse_dates=["start_date"])
season_df = season_df.sort_values("season_year").reset_index(drop=True)
season_df["next_start"] = season_df["start_date"].shift(-1).fillna(pd.Timestamp("2100-01-01"))

In [37]:
season_df

,season_year,start_date,next_start
0,1960,1960-10-01,1961-10-01
1,1961,1961-10-01,1962-10-01
2,1962,1962-10-01,1963-10-01
3,1963,1963-10-01,1964-10-01
4,1964,1964-10-01,1965-10-01
...,...,...,...
61,2021,2021-10-01,2022-10-01
62,2022,2022-10-01,2023-10-01
63,2023,2023-10-01,2024-10-01
64,2024,2024-10-01,2025-10-01


In [39]:
records = []
for _, s in season_df.iterrows():
    year       = s["season_year"]
    start      = s["start_date"]
    next_start = s["next_start"]

    # a) events before this season
    prev = inj[inj["injury_date"] < start]
    counts = prev.groupby("player_name")[[f"bp_{p}" for p in body_parts] +
                                         [f"it_{t}" for t in injury_types]].sum()
    last   = prev.groupby("player_name")["injury_date"].max().rename("latest_injury")

    # b) up to next season to flag still injured
    upto_next = inj[inj["injury_date"] < next_start].sort_values("injury_date")
    last_note = upto_next.groupby("player_name").tail(1).set_index("player_name")["notes"]
    still_on_il = (last_note.str.contains("placed on il",   case=False, na=False) &
                   ~ last_note.str.contains("activated", case=False, na=False))
    still_on_il = still_on_il.rename("still_injured")

    # c) assemble and compute days-before-last-injury
    df = (
        counts
        .join(last, how="outer")
        .join(still_on_il, how="outer")
        .reset_index()
    )
    df["season_start"]                = start
    df["days_before_last_injury"]     = (start - df["latest_injury"]).dt.days
    df["season"]                      = year
    records.append(df)

summary = pd.concat(records, ignore_index=True)

In [41]:
del summary['season_start']

In [43]:
# Using lstrip
summary['player_name'] = summary['player_name'].str.lstrip(' ')

In [45]:
# 7. Save or inspect
summary.to_csv("player_season_injury_summary.csv", index=False)

In [47]:
summary[(summary['bp_Ankle'] == 1) & (summary['season'] == 2021)].tail(235)

,player_name,bp_Ankle,bp_Knee,bp_Foot,bp_Calf,bp_Hamstring,bp_Quadriceps,bp_Achilles,bp_Toe,bp_Shoulder,...,it_Stinger,it_Contusion,it_Concussion,it_Soreness,it_Inflammation,it_Surgery,latest_injury,still_injured,days_before_last_injury,season
50553,Joey Graham,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2011-02-16,False,3880.0,2021
50554,Joffrey Lauvergne,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2017-11-14,False,1417.0,2021
50557,John Bagley,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1990-01-06,False,11591.0,2021
50574,John Lucas III,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2017-01-06,False,1729.0,2021
50592,John Wallace,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2010-12-26,False,3932.0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51943,Willie Burton,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1998-04-20,False,8565.0,2021
51945,Willie Green,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2014-12-23,False,2474.0,2021
51954,Xavier Henry,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2014-04-06,True,2735.0,2021
51962,Yao Ming,1.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,2010-11-12,True,3976.0,2021


In [105]:
import difflib

# your target name
target = "Kawhi Leonard"

# get unique player names from the DataFrame
names = summary["player_name"].unique().tolist()

# find the single closest match
best_match = difflib.get_close_matches(target, names, n=1, cutoff=0)[0]

print("Closest name to", target, "is", best_match)

Closest name to Kawhi Leonard is  Kawhi Leonard
